In [3]:
import numpy as np
from numpy.linalg import solve
import math
from sympy import Plane, Point3D, Line3D

In [4]:
def read_input(fname):
    data = []
    with open(fname, 'r') as inf:
        for line in inf.readlines():
            if line.strip() == '':
                continue
            p, s = line.strip().split(' @ ')
            data.append((tuple(int(v) for v in p.split(',')), tuple(int(v) for v in s.split(','))))

    return data

In [5]:
def n_crossing(data, minv, maxv):
    n = 0
    for i, (p, r) in enumerate(data):
        for j in range(i+1, len(data)):
            p = np.asarray(p[0:2])
            r = np.asarray(r[0:2])
        
            q = np.asarray(data[j][0][0:2])
            s = np.asarray(data[j][1][0:2])

            # print(p, q, r, s)
            c1 = np.cross(q-p, s)
            c2 = np.cross(r, s)

            if c2 == 0:
                print('Parallel!', i, j)
                continue

            t = c1/c2

            res = p +t*r
            # print(res, t)

            dist1 = res-p
            dist2 = res-q
            if minv<=res[0]<=maxv and minv<=res[1]<=maxv:
                if np.all(np.sign(dist1) == np.sign(r)) and np.all(np.sign(dist2) == np.sign(s)):
#                     print('y', res, p, r, q, s, t)
                    n += 1
#                 else:
#                     print(res, p, r, q, s, t)

    return n

In [19]:
def hail_hitter(data, first, second, third):

    A = np.array((6, 6))
    B = np.array(6)

    (p1x, p1y, p1z), (d1x, d1y, d1z) = data[first]
    (p2x, p2y, p2z), (d2x, d2y, d2z) = data[second]
    (p3x, p3y, p3z), (d3x, d3y, d3z) = data[third]
    A = np.array([[d1y-d2y, d2x-d1x, 0, p2y-p1y, p1x-p2x, 0],
                  [d1y-d3y, d3x-d1x, 0, p3y-p1y, p1x-p3x, 0],
                  [d1z-d2z, 0, d2x-d1x, p2z-p1z, 0, p1x-p2x],
                  [d1z-d3z, 0, d3x-d1x, p3z-p1z, 0, p1x-p3x],
                  [0, d1z-d2z, d2y-d1y, 0, p2z-p1z, p1y-p2y],
                  [0, d1z-d3z, d3y-d1y, 0, p3z-p1z, p1y-p3y]])
    B = np.array([p2y*d2x - p2x*d2y - p1y*d1x + p1x*d1y,
                  p3y*d3x - p3x*d3y - p1y*d1x + p1x*d1y,
                  p2z*d2x - p2x*d2z - p1z*d1x + p1x*d1z,
                  p3z*d3x - p3x*d3z - p1z*d1x + p1x*d1z,
                  p2z*d2y - p2y*d2z - p1z*d1y + p1y*d1z,
                  p3z*d3y - p3y*d3z - p1z*d1y + p1y*d1z])

    X = solve(A, B)

    print(X)

    return np.round(np.sum(X[0:3]))

In [20]:
parallels = find_parallels(read_input('input24a.txt'))
for idx, (i,j) in enumerate(parallels[:-1]):
    for idx2 in range(idx+1, len(parallels)):
        i, j = parallels[idx2]
        # print(f'Comparing {(i,j)} with {(h,k)}')
        hail_hitter2(read_input('input24a.txt'), i, j, 0, 4)
        # print(hail_hitter2(read_input('input24.txt'), i, j, 99, 135))

NameError: name 'find_parallels' is not defined

In [22]:
hail_hitter(read_input('input24.txt'), 0, 1, 2)
# hail_hitter2(read_input('input24.txt'), 14, 159, 0, 4)

[ 3.09991771e+14  4.60585296e+14  2.34197929e+14 -6.30000000e+01
 -3.01000000e+02  9.70000000e+01]


1004774995964534.0

In [293]:
print(n_crossing(read_input('input24a.txt'), 7, 27))
print(n_crossing(read_input('input24.txt'), 200000000000000, 400000000000000))


Parallel! 1 2
2
Parallel! 14 159
Parallel! 23 35
Parallel! 31 265
Parallel! 110 125
Parallel! 139 152
16018


In [292]:
969676900505341 > 787702817644458

True

In [358]:
def find_parallels(data):
    p = []
    for i in range(0, len(data)-1):
        for j in range(i+1, len(data)):
            # print(set(np.round(np.array(data[i][1]) / np.array(data[j][1]), 10)))
            if len(set(np.abs(np.array(data[i][1]) / np.array(data[j][1])))) <= 2:
                print(f'{i} and {j} are parallel, {data[i][1]} - {data[j][1]}')
                p.append((i, j))
    return p
            

In [409]:
def inspect(data):
    for i in range(0, len(data)-1):
        p1, d1 = data[i]
        l1_p1 = Point3D(*p1)
        l1_p2 = Point3D(p1[0]+d1[0], p1[1]+d1[1], p1[2]+d1[2])
        l1 = Line3D(l1_p1, l1_p2)
        for j in range(i+1, len(data)):
            p2, d2 = data[j]
            l2_p1 = Point3D(*p2)
            l2_p2 = Point3D(p2[0]+d2[0], p2[1]+d2[1], p2[2]+d2[2])
            l2 = Line3D(l2_p1, l2_p2)
            if l1.is_parallel(l2):
                print(i, j, 'parallel')
            if len(l1.intersection(l2)) != 0:
                print(i, j, 'intersect')
            if l1.is_perpendicular(l2):
                print(i, j, 'perp', p1, d1, p2, d2)
            if l1.is_similar(l2):
                print(i, j, 'sim')
            print(i, j, float(l1.smallest_angle_between(l2))*180/math.pi)
    return p


In [410]:
inspect(read_input('input24.txt'))

0 1 45.69605998573408
0 2 80.24320097323829
0 3 60.351437676537785
0 4 4.989859719264904
0 5 88.93656005276253
0 6 43.343002694964305
0 7 72.80753235361071
0 8 61.86422727440409
0 9 61.18195348954555
0 10 26.589567357157847
0 11 24.749723323694134
0 12 53.96747792809583
0 13 61.407526426470625
0 14 21.307533182233026
0 15 76.07870771553478
0 16 46.75531337910303
0 17 75.75739868101259
0 18 41.42290903785073
0 19 77.78939007738644
0 20 65.34603900665411
0 21 66.70860621382354
0 22 70.06896291185046
0 23 83.66108128351745
0 24 60.88437131487184
0 25 67.80100847320165
0 26 35.60002814161993
0 27 81.72242613914243
0 28 38.832261840351535
0 29 53.61620411844672
0 30 73.08915723733443
0 31 76.57156684577436
0 32 35.18646103599901
0 33 48.053677841670286
0 34 52.04267257258832
0 35 51.144764169523434
0 36 47.943484884972534
0 37 27.864361027877955
0 38 73.80262770604257
0 39 28.416063240249123
0 40 85.4007963215891
0 41 13.920645545058267
0 42 47.648292949238936
0 43 66.78542057405289
0 44 79

array([19, 13])

In [400]:
hail_hitter2(read_input('input24.txt'), 63, 139, 0, 4)

T Point3D(343570910546696, 233332531101051, 218256317021745), Line3D(Point3D(343570910546696, 233332531101051, 218256317021745), Point3D(343570910546534, 233332531101420, 218256317021889)), Point3D(343570910546696, 233332531101051, 218256317021745), Point3D(343570910546534, 233332531101420, 218256317021889)
Line1: Point3D(246123428675951, 99127506336088, 302231597481979), Point3D(246123428683451, 99127506384088, 302231597476979), Line3D(Point3D(246123428675951, 99127506336088, 302231597481979), Point3D(246123428683451, 99127506384088, 302231597476979))
Inter Point3D(71373762738186043431097611007/294376095191831, 106588265457132803322903660851/588752190383662, 155932333132709421182591475307/588752190383662) Point3D(2998774532525907082912687056209/11539450246018609, 2159207954194850699127922079512/11539450246018609, 3381822833291179279384702624511/11539450246018609)
Point_p Point3D(71373762738186043431097611007/294376095191831, 106588265457132803322903660851/588752190383662, 155932333132

In [401]:
hail_hitter2(read_input('input24.txt'), 177, 195, 0, 4)

T Point3D(305032967853318, 421048896242136, 251754771047249), Line3D(Point3D(305032967853318, 421048896242136, 251754771047249), Point3D(305032967853292, 421048896242130, 251754771047215)), Point3D(305032967853318, 421048896242136, 251754771047249), Point3D(305032967853292, 421048896242130, 251754771047215)
Line1: Point3D(246123428675951, 99127506336088, 302231597481979), Point3D(246123428683451, 99127506384088, 302231597476979), Line3D(Point3D(246123428675951, 99127506336088, 302231597481979), Point3D(246123428683451, 99127506384088, 302231597476979))
Inter Point3D(13375355038895083425317733484427/42831561578274759, 19954194565224230145971218996579/42831561578274759, 10290559173209964849597367805776/42831561578274759) Point3D(302980566698960816927144610366/1082597922137261, 341093499426595635119264335960/1082597922137261, 908530189864490717429169958247/3247793766411783)
Point_p Point3D(13375355038895083425317733484427/42831561578274759, 19954194565224230145971218996579/428315615782747